# Project 1 Code

In [1]:
import pulp
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
import pandas as pd
from openpyxl import load_workbook

def writeSolution(solution, solution_type, file_path, row_names, col_names, start_row):
    """
    This function writes the solution to the original data file in Excel.
    """
    df = pd.DataFrame(solution, index=row_names, columns=col_names)

    # Step 1: Write DataFrame to Excel
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
        df.to_excel(writer, startrow=start_row, startcol=0)

    # Step 2: Modify the Excel file with openpyxl
    wb = load_workbook(file_path)  # Load the workbook
    ws = wb.active  # Select the active sheet

    # Write extra value at C5
    ws["A"+ str(start_row+1)] = solution_type 
    
    # Write solution_type at the start_row in column 1 (A column)
    # ws.cell(row=start_row, column=1, value=solution_type)

    # Save the modified Excel file
    wb.save(file_path)

    return df.astype(int).style.set_caption(solution_type)


In [3]:
def importDemand(file_path):
    """
    This function imports demand from Excel file 
    """
    # Read the entire sheet first
    df = pd.read_excel(file_path, header=None)  # Read without headers to get raw data
    
    # Extract the range (B11:H16) → (row 10 to 15, col 1 to 7) using .iloc
    demand = df.iloc[10:16, 1:8].values  # Convert to NumPy array
    
    return demand

In [4]:
file_path = 'Project1Data.xlsx'

In [5]:
numproducts = 6
numdays = 7

#Begin by setting up your sets for the i and j indices
products = []
for i in range (0,numproducts):
    products.append(i)

days =[]
for j in range (0,numdays):
    days.append(j)

### Add in the parameters

In [6]:

sellprice    = [3000,3500,2800,4000,2400,3000]
mfgcost      = [100,300,200,600,50,250]
materialcost = 2000
inventorycost = 20

namesproduct=['Nails','Screws', 'Pipe', 'Flashing', 'Rebar', 'Conduit']
namesday=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

demands = importDemand(file_path)
# print('Demands = \n', demands)


## Linear Programming

In [7]:
#Then instantiate a problem class, we’ll name it “My LP problem” and we’re looking for an optimal maximum so we use LpMaximize

my_lp_problem = pulp.LpProblem("My LP Problem", pulp.LpMaximize)

#Set up the variables

produced = pulp.LpVariable.dicts("amtproduced", ((i, j) for i in products for j in days), lowBound=0, cat = 'Integer')

inventory = pulp.LpVariable.dicts("amtinventory", ((i, j) for i in products for j in days), lowBound=0, cat = 'Integer')

backorder = pulp.LpVariable.dicts("amtbackorder", ((i, j) for i in products for j in days), lowBound=0, cat = 'Integer')

sold      = pulp.LpVariable.dicts("amtsold", ((i, j) for i in products for j in days), lowBound=0, cat = 'Integer')

# obj and constraints are added using += to the my_lp_problem class.
# Pulp assumes that the the objective function is always given first. 

# Objective
my_lp_problem += pulp.lpSum([sold[i,j]*sellprice[i] for i in products for j in days]) \
                - pulp.lpSum([produced[i,j]*mfgcost[i] for i in products for j in days]) \
                - inventorycost*pulp.lpSum([inventory[i,j] for i in products for j in days]) \
                - pulp.lpSum([backorder[i,j]*0.02*sellprice[i] for i in products for j in days]) \
                - materialcost*pulp.lpSum([produced[i,j] for i in products for j in days ])
# Contraints
## Contraint 1: Each day, the company can process up to 500 tons of steel.  
for j in range(0, len(days)):
    my_lp_problem += pulp.lpSum([produced[i,j] for i in products]) <= 500

## Contraint 2: Can't ship out/sell more than demand
for i in range(0, len(products)):
    for j in range(0, len(days)):
        my_lp_problem += sold[i,j]  <= demands[i][j] 


## Constraint 3a: Flow In = Flow Out for Day 1-6
for i in range(0, len(products)):
    for j in range(1, len(days)):
        my_lp_problem += produced[i,j] + inventory[i,j-1] + backorder[i,j] == sold[i,j] + inventory[i,j] + backorder[i,j-1]

## Constraint 3b: Flow In = Flow Out for Day 0
for i in range(0, len(products)):
    my_lp_problem += produced[i,0] + backorder[i,0] == sold[i,0] + inventory[i,0]

## Constraint 4: No ending inventory
for i in range(0, len(products)):
    my_lp_problem += inventory[i,6] == 0

## Constraint 5: No backlog at the end of the week
for i in range(0, len(products)):
    my_lp_problem += backorder[i,6] == 0


print (my_lp_problem)
pulp.apis.COIN_CMD(gapAbs=.0001)
status=my_lp_problem.solve()
if pulp.LpStatus[my_lp_problem.status] =='Infeasible':
    print ('INFEASIBLE ****************************************')
elif pulp.LpStatus[my_lp_problem.status] =='Unbounded':
    print ('Unbounded ****************************************')

else:
    #This is various ways to display the solution
    for variable in my_lp_problem.variables():
         print ("{} = {}".format(variable.name, variable.varValue))
    print ('\nFeasible\n\nThe objective value is ', pulp.value(my_lp_problem.objective),'\n')
    
    #This only displays non 0s
    for variable in my_lp_problem.variables():
        if variable.varValue>.000001 or variable.varValue<-.00001:
            print ("{} = {}".format(variable.name, variable.varValue))
    print ('\nFeasible\n\nThe objective value is ', pulp.value(my_lp_problem.objective),'\n\n')

/opt/anaconda3/lib/python3.12/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


My_LP_Problem:
MAXIMIZE
-60.0*amtbackorder_(0,_0) + -60.0*amtbackorder_(0,_1) + -60.0*amtbackorder_(0,_2) + -60.0*amtbackorder_(0,_3) + -60.0*amtbackorder_(0,_4) + -60.0*amtbackorder_(0,_5) + -60.0*amtbackorder_(0,_6) + -70.0*amtbackorder_(1,_0) + -70.0*amtbackorder_(1,_1) + -70.0*amtbackorder_(1,_2) + -70.0*amtbackorder_(1,_3) + -70.0*amtbackorder_(1,_4) + -70.0*amtbackorder_(1,_5) + -70.0*amtbackorder_(1,_6) + -56.0*amtbackorder_(2,_0) + -56.0*amtbackorder_(2,_1) + -56.0*amtbackorder_(2,_2) + -56.0*amtbackorder_(2,_3) + -56.0*amtbackorder_(2,_4) + -56.0*amtbackorder_(2,_5) + -56.0*amtbackorder_(2,_6) + -80.0*amtbackorder_(3,_0) + -80.0*amtbackorder_(3,_1) + -80.0*amtbackorder_(3,_2) + -80.0*amtbackorder_(3,_3) + -80.0*amtbackorder_(3,_4) + -80.0*amtbackorder_(3,_5) + -80.0*amtbackorder_(3,_6) + -48.0*amtbackorder_(4,_0) + -48.0*amtbackorder_(4,_1) + -48.0*amtbackorder_(4,_2) + -48.0*amtbackorder_(4,_3) + -48.0*amtbackorder_(4,_4) + -48.0*amtbackorder_(4,_5) + -48.0*amtbackorder_(4,_6

### Extract Data

In [16]:


soln_product = np.zeros((len(products), len(days))) 
soln_inventory = np.zeros((len(products), len(days))) 
soln_backorder = np.zeros((len(products), len(days))) 
soln_sold = np.zeros((len(products), len(days))) 

for i in range(len(products)):
    for j in range(len(days)):
        soln_product[i, j] = int(produced[i, j].varValue ) 
        soln_inventory[i, j] = inventory[i, j].varValue  
        soln_backorder[i, j] = backorder[i, j].varValue  
        soln_sold[i, j] = sold[i, j].varValue  

print('Produced \n',soln_product,'\n')
print('Inventory \n',soln_inventory,'\n')
print('Back order \n',soln_backorder,'\n')
print('Sold \n',soln_sold,'\n')
print('Demand \n',demands,'\n')

Produced 
 [[ 50.  40.  60.  90.   0.  30.  30.]
 [ 90.  70.  80.  70.  70.  40.  40.]
 [ 90.  60. 125.   5.  80.  40.  30.]
 [ 70.  60.  80. 160.   0.  60.  30.]
 [ 50. 270.  80. 175. 200.  75.  75.]
 [150.   0.   0.   0. 150.   0.   0.]] 

Inventory 
 [[  0.   0.   0.  50.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  55.   0.   0.   0.   0.]
 [  0.   0.   0. 120.   0.   0.   0.]
 [  0.   0.   0. 100.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]] 

Back order 
 [[  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [200.   5.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]] 

Sold 
 [[ 50.  40.  60.  40.  50.  30.  30.]
 [ 90.  70.  80.  70.  70.  40.  40.]
 [ 90.  60.  70.  60.  80.  40.  30.]
 [ 70.  60.  80.  40. 120.  60.  30.]
 [250.  75.  75.  75. 300.  75.  75.]
 [150.   0.   0.   0. 150.   0.   0.]] 

Demand 
 [[50 40 60 40 50 30 30]


## Sanity check

In [17]:
revenue = np.zeros(len(products))
for i in range(len(products)):
    for j in range(len(days)):
        revenue[i] += sold[i,j].varValue*sellprice[i] - produced[i,j].varValue*mfgcost[i] - 20*inventory[i,j].varValue \
                           - backorder[i,j].varValue*0.02*sellprice[i] - materialcost*produced[i,j].varValue
print((revenue))
print(np.sum(revenue)) 

[269000. 552000. 256900. 641600. 311910. 225000.]
2256410.0


## Presenting tables

In [18]:
writeSolution(soln_sold,'Sold', file_path, namesproduct, namesday, 30)
writeSolution(soln_inventory,'Inventory', file_path, namesproduct, namesday, 39)
writeSolution(soln_backorder,'Backorder', file_path, namesproduct, namesday, 48)
writeSolution(soln_product,'Production', file_path, namesproduct, namesday, 21)

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Nails,50,40,60,90,0,30,30
Screws,90,70,80,70,70,40,40
Pipe,90,60,125,5,80,40,30
Flashing,70,60,80,160,0,60,30
Rebar,50,270,80,175,200,75,75
Conduit,150,0,0,0,150,0,0


In [19]:
writeSolution(soln_sold,'Sold', file_path, namesproduct, namesday, 30) 

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Nails,50,40,60,40,50,30,30
Screws,90,70,80,70,70,40,40
Pipe,90,60,70,60,80,40,30
Flashing,70,60,80,40,120,60,30
Rebar,250,75,75,75,300,75,75
Conduit,150,0,0,0,150,0,0


In [20]:
writeSolution(soln_inventory,'Inventory', file_path, namesproduct, namesday, 39) 

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Nails,0,0,0,50,0,0,0
Screws,0,0,0,0,0,0,0
Pipe,0,0,55,0,0,0,0
Flashing,0,0,0,120,0,0,0
Rebar,0,0,0,100,0,0,0
Conduit,0,0,0,0,0,0,0


In [21]:
writeSolution(soln_backorder,'Backorder', file_path, namesproduct, namesday, 48) 

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Nails,0,0,0,0,0,0,0
Screws,0,0,0,0,0,0,0
Pipe,0,0,0,0,0,0,0
Flashing,0,0,0,0,0,0,0
Rebar,200,5,0,0,0,0,0
Conduit,0,0,0,0,0,0,0
